In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urlparse, parse_qs


def sanitize_filename(name):
    """
    OS에서 사용할 수 없는 문자 제거
    """
    return re.sub(r'[\\/:*?"<>|]', '_', name)


def get_webtoon_title_and_episode(webtoon_url):
    """
    웹툰 상세 페이지에서 제목과 회차 번호 추출
    """
    res = requests.get(webtoon_url)
    res.encoding = 'utf-8'
    if not res.ok:
        print(f"[에러] 웹툰 페이지 요청 실패: {res.status_code}")
        return None, None, None

    soup = BeautifulSoup(res.text, 'html.parser')

    # 제목 추출: <div class="title_area"><strong>...</strong>
    strong_tag = soup.select_one('div.title_area strong')
    if strong_tag:
        full_title = strong_tag.text.strip()
        title = full_title.split('-')[0].strip()  # "낢이 사는 이야기 - 부제목" → "낢이 사는 이야기"
        title = sanitize_filename(title)
    else:
        title = "제목없음"

    # 회차 번호 추출: URL의 no 파라미터
    parsed_url = urlparse(webtoon_url)
    no_param = parse_qs(parsed_url.query).get('no', ['0'])[0]
    episode = f"{no_param}화"

    return soup, title, episode


def get_image_urls(soup):
    """
    BeautifulSoup 객체에서 본문 이미지 URL 목록 추출
    """
    img_tags = soup.select("img[src*='IMAG01']")
    img_urls = [img['src'] for img in img_tags]
    return img_urls


def create_directory(path):
    """
    디렉토리 생성 (중첩 포함)
    """
    os.makedirs(path, exist_ok=True)


def download_images(img_urls, save_dir, referer_url):
    """
    이미지 다운로드 및 저장
    """
    headers = {'referer': referer_url}

    for img_url in img_urls:
        res = requests.get(img_url, headers=headers)
        if not res.ok:
            print(f"[에러] 이미지 요청 실패: {res.status_code} | {img_url}")
            continue

        file_name = os.path.basename(img_url)
        file_path = os.path.join(save_dir, file_name)

        with open(file_path, 'wb') as f:
            f.write(res.content)
            print(f"[저장 완료] {file_path} ({len(res.content):,} bytes)")


def main():
    # ▶️ 웹툰 상세 페이지 URL 입력
    webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=48&week=tue'

    # 제목, 회차, soup 추출
    soup, title, episode = get_webtoon_title_and_episode(webtoon_url)
    if not soup or not title or not episode:
        print("[중단] 제목 또는 회차 정보를 가져오지 못했습니다.")
        return

    # 디렉토리 생성: img/제목/회차/
    save_dir = os.path.join("img", title, episode)
    create_directory(save_dir)

    print(f"[시작] {title} {episode} 이미지 다운로드...")

    # 이미지 URL 추출
    img_urls = get_image_urls(soup)
    if not img_urls:
        print("[중단] 이미지가 없습니다.")
        return

    # 이미지 다운로드
    download_images(img_urls, save_dir, referer_url=webtoon_url)

    print(f"[완료] {title} {episode} 이미지 다운로드 완료 ✅")


if __name__ == '__main__':
    main()

## 특정 웹툰 페이지의 모든 image를 다운로드 받기
- img 폴더를 생성하고 그 아래에 파일을 저장하기
- pathlib 의 Path 사용하여 디렉토리 생성하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from pathlib import Path

def download_one_episode(title,no,url):
    
    req_header = {'referer': url}
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
               
        img_tags = soup.select("img[src*='IMAG01']")
        imgurl_list = [img['src'] for img in img_tags]
        print(len(imgurl_list))
        
        save_dir = Path('img') / title / str(no)
        save_dir.mkdir(parents=True, exist_ok=True)

        for idx,img_url in enumerate(imgurl_list,1):
            res = requests.get(img_url,headers=req_header)
            res.raise_for_status()

            file_name = Path(img_url).name
            save_path = save_dir / file_name
            save_path.write_bytes(res.content)
            print(f'다운로드 완료: {save_path} ({save_path.stat().st_size:,} bytes)')
                
if __name__ == '__main__':                
    download_one_episode('낢이사는이야기',47,'https://comic.naver.com/webtoon/detail?titleId=833255&no=47&week=tue')

## 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기
- 첫번째 Page의 회차의 url를 가져오기 ( image 다운로드는 호출 않함 )

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]

    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}&page=1'
               #https://comic.naver.com/webtoon/detail?titleId=826419&no=46
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok:
        #pprint(res.json()['articleList'])
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')

In [ ]:
from urllib.parse import urlparse, parse_qs

"""url에 titleId를 반환하는 함수"""
def get_title_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    return title_id

#테스트 하기
url = 'https://comic.naver.com/webtoon/list?titleId=826419'
print(get_title_id(url))  # 출력: 826419

In [ ]:
def calculate_pages(total_items, items_per_page=20):
    """총 페이지 수 계산 함수"""
    return (total_items + items_per_page - 1) // items_per_page

# 예제 사용
total_items = 49
items_per_page = 20

total_pages = calculate_pages(total_items, items_per_page)
print(f"총 {total_items}개의 항목을 {items_per_page}개씩 출력할 때 필요한 페이지 수: {total_pages}")
# 출력: 총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3

## 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기
- 모든 Page의 회차 url를 가져오기 ( image 다운로드는 호출 안함 )

In [ ]:
import requests

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            page = count + 1
            req_param = { "page": page}
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')

## image download 처리하기
- 모든 Page의 회차 url를 가져오기 ( image 다운로드는 호출함 )

In [ ]:
import requests
from time import sleep

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            page = count + 1
            req_param = { "page": page}
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
                download_one_episode(title,no,detail_url)
                #0.5초간 프로세스를 중지함, 기계가 아니라 사람처럼 보이게 하려고
                sleep(0.5)

if __name__ == '__main__': 
    #download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')
    download_all_episode('냉동무사','https://comic.naver.com/webtoon/list?titleId=836370')